<a href="https://www.kaggle.com/code/joshuaokolo/covid-19-dna-eda?scriptVersionId=105424522" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

EDA Sample

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import Bio
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd

In [ ]:
ncov_record = SeqIO.read("/kaggle/input/covid19-genome-sequence/sequence.fasta", "fasta")

In [ ]:
ncov_dna = ncov_record.seq

## Transcription

In [ ]:
#Protein Synthesis
ncov_mrna = ncov_dna.transcribe()
ncov_mrna

## Translation

In [ ]:
ncov_protein = ncov_mrna.translate()
ncov_protein

## Number of Proteins

In [ ]:
len(ncov_protein)

## Deriving Amino Acids from Protein Structures

In [ ]:
ncov_aa = ncov_protein.split("*")
ncov_aa[0:10]

## Clean List of Proteins

In [ ]:
ncov_clean = [ str(i) for i in ncov_aa]

In [ ]:
#Identify all the Proteins (chains of amino acids)
Proteins = ncov_protein.split('*')
Proteins[0:10]

## Remove chains shorter than 20 amino acids long

In [ ]:
#Remove chains smaller than 20 amino acids long
for i in Proteins[:]:
    if len(i) < 20:
        Proteins.remove(i)

## Protein Analysis

In [ ]:
df = pd.DataFrame({"amino_acids":ncov_clean})

In [ ]:
df['count'] = df['amino_acids'].str.len()

In [ ]:
df.nlargest(10, "count")

In [ ]:
#protein of interest
soi=df['amino_acids'].iloc[705]
seq_oi=Seq(soi)
seq_oi

In [ ]:
#cleaning
soi1=''
for i in soi:
    if(i!='X'):
        soi1=soi1+i
print(len(soi1))

In [ ]:
a=ProteinAnalysis(soi1)
a

In [ ]:
MW=a.molecular_weight()
MW

In [ ]:
aromaticity=a.aromaticity()
aromaticity

In [ ]:
IsoElectric=a.isoelectric_point()
IsoElectric

In [ ]:
def amino_acid_composition(protein_seq):
    amino_acids_dict = {'Y': 0, 'A': 0, 'Q': 0, 'D': 0, 'C': 0, 'G': 0, 'V': 0, 'T': 0, 'E': 0, 'N': 0, 
                       'K': 0, 'R': 0, 'S': 0, 'I': 0, 'H': 0, 'M': 0, 'F': 0, 'L': 0, 'W': 0, 'P': 0}
    for amino_acid in amino_acids_dict:
        amino_acids_dict[amino_acid] = protein_seq.count(amino_acid)/len(protein_seq)*100
    return amino_acids_dict

In [ ]:
ac = amino_acid_composition(ncov_protein)
amc = pd.DataFrame.from_dict(ac, orient ='index')
amc = amc.reset_index()
amc = amc.rename(columns={"index": "amino_acid", 0: "composition"})

In [ ]:
r = amc.sort_values(by='composition', ascending=False)
r.style.bar(subset=["composition"],color='#').background_gradient(cmap='Purples')

In [ ]:
ax = sns.barplot(x="amino_acid", y="composition", data=r,
                 palette="Blues_d")

In [ ]:
#ORF
orf_strands = []
# generate ORFs
for seq in ncov_protein.split("STOP"):
    for nu in range(len(seq)-1, -1, -1):
        if seq[nu] == "M":
            orf_strands.append(seq[nu:]+"STOP")

orf_strands[0:10]

In [ ]:
len(orf_strands)
#Conclusion: This is how I got Started with BioPython Packages